In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import findspark
findspark.init("/usr/local/spark/spark-2.4.4-bin-hadoop2.7")
import pyspark as spark
from pyspark.sql.functions import udf,col, rand
from pyspark.sql.types import StringType

In [2]:
spark.SparkConf()

In [3]:
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [4]:
df = sql.read.option("delimiter",'\t').option("header",'true').csv('GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct')
df = df.withColumn('ensg',udf(lambda x: x[:15], StringType())(col('Name')))
df.registerTempTable('gtex')

In [5]:
genes = sql.read.option("header",'true').csv('../genes.txt')
genes.registerTempTable("genes")
genes.select("type_of_gene").distinct().show()

+--------------+
|  type_of_gene|
+--------------+
|          rRNA|
|         ncRNA|
|       unknown|
|         scRNA|
|      microRNA|
|         snRNA|
|     antisense|
|        snoRNA|
|        pseudo|
|protein-coding|
|    intergenic|
+--------------+



In [6]:
pc = sql.sql("SELECT * FROM genes WHERE type_of_gene='protein-coding'")
pc_list = list(pc.select('_c0').orderBy(rand()).toPandas().values.T[0])

In [7]:
df = df.filter(col("ensg").isin(pc_list))

In [ ]:
df.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("mainTable.csv")

In [ ]:
#df.toPandas().to_csv("mainTable.csv", index=True, header=True)

## GTEx Biospecimen
[GTEX_biospecimen](https://gtexportal.org/home/samplesPage)

In [8]:
#df_file = sql.createDataFrame(pd.read_csv("https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SampleAttributesDS.txt", sep='\t').loc[:,['SAMPID','SMTS', 'SMTSD']])
df_file=sql.read.option('delimiter',',').option("header",'true').csv("files.dat")
df_file.registerTempTable("files")
df_file.describe()

DataFrame[summary: string, sample: string, primary_site: string, secondary_site: string]

In [9]:
#df_sites=pd.DataFrame()
#df_sites.insert(0,'sample',df_file.select('SAMPID').toPandas().values.T[0])
#df_sites.insert(1,'primary_site',df_file.select('SMTS').toPandas().values.T[0])
#df_sites.insert(2,'secondary_site', df_file.select('SMTSD').toPandas().values.T[0])
#df_sites.set_index('sample', inplace=True)
#df_sites.head()
#df_sites.to_csv("files.dat", index=True, header=True)

In [10]:
df_file.select('primary_site').distinct().show()

+---------------+
|   primary_site|
+---------------+
|           Skin|
|  Adrenal Gland|
| Adipose Tissue|
|      Pituitary|
| Salivary Gland|
|         Spleen|
|          Colon|
|          Nerve|
|          Blood|
|         Kidney|
|   Blood Vessel|
|         Muscle|
|       Prostate|
|Small Intestine|
|      Esophagus|
|        Bladder|
|          Liver|
|        Stomach|
|        Thyroid|
|          Heart|
+---------------+
only showing top 20 rows



In [11]:
available_tissues=df_file.groupby('primary_site').count().sort("count", ascending=False).toPandas()
available_tissues

,primary_site,count
0,Blood,2561
1,Brain,2076
2,Skin,1362
3,Esophagus,1111
4,Blood Vessel,1041
5,Adipose Tissue,886
6,Heart,727
7,Muscle,718
8,Lung,607
9,Thyroid,564


In [14]:
#selectedTissues = ['Blood', 'Brain', 'Lung', 'Thyroid', 'Breast', 'Skin', 'Pancreas', 'Testis', 'Adipose Tissue', 'Colon', 'Nerve', 'Heart', 'Kidney', 'Uterus', 'Adrenal Gland']
selectedTissues = available_tissues['primary_site'][:].values
selectedTissues

array(['Blood', 'Brain', 'Skin', 'Esophagus', 'Blood Vessel',
       'Adipose Tissue', 'Heart', 'Muscle', 'Lung', 'Thyroid', 'Colon',
       'Nerve', 'Breast', 'Testis', 'Stomach', 'Pancreas',
       'Adrenal Gland', 'Pituitary', 'Liver', 'Spleen', 'Bone Marrow',
       'Prostate', 'Small Intestine', 'Ovary', 'Vagina', 'Uterus',
       'Salivary Gland', 'Kidney', 'Bladder', 'Cervix Uteri',
       'Fallopian Tube'], dtype=object)

In [63]:
limit=int(1000/len(selectedTissues))
needed_columns = []
for tissue in selectedTissues:
    needed_columns = np.concatenate((needed_columns,
                                 list(set(sql.sql("SELECT * FROM files WHERE primary_site='%s'"%(tissue))
                                .orderBy(rand())
                                .limit(limit)
                                .select("sample")
                                .toPandas().values.T[0])-{'Name'}-{'Description'})))
needed_columns=needed_columns[np.isin(needed_columns,df.columns)]
np.random.shuffle(needed_columns)
needed_columns = np.concatenate([['ensg'],needed_columns])

In [24]:
needed_columns = np.concatenate([['ensg'],df.columns[2:-1]])
needed_columns

array(['ensg', 'GTEX-1117F-0226-SM-5GZZ7', 'GTEX-111CU-1826-SM-5GZYN',
       ..., 'GTEX-ZVZQ-0006-SM-51MR8', 'GTEX-ZXES-0005-SM-57WCB',
       'GTEX-ZXG5-0005-SM-57WCN'], dtype='<U30')

In [27]:
df.select(list(needed_columns)).repartition(1).write.format("com.databricks.spark.csv").mode('overwrite').option("header", "true").save("mainTable_all.csv", header=True, index=False)

In [29]:
df_file.filter(col('sample').isin(list(needed_columns))).toPandas().to_csv("files_tissues.dat", header=True, index=False)

# U

In [ ]:
tissue = 'Breast'

In [ ]:
sample_list = df_sites[df_sites['primary_site']==tissue].index.values
subdf = df.loc[:,sample_list]

In [ ]:
odf = pd.read_csv("meanVariances.csv", header=0, index_col=0)
odf.insert(3,"A",pd.read_csv("A.dat", header=None).values)
odf.head()

In [ ]:
O = odf[odf.index.isin(pc.index)]['occurrence']
O_nc = odf[~odf.index.isin(pc.index)]['occurrence']

In [ ]:
bins = 20
rang = (0-0.5/20,1+0.5/20)
fig = plt.figure()
plt.hist(O[O<=1], histtype='step', lw=4, density=True, bins=bins, range=rang, label='coding')
plt.hist(O_nc[O_nc<=1], histtype='step', lw=4, density=True, bins=bins, range=rang, label='non-coding')
#plt.title(tissue, fontsize=18)
plt.xlabel('$O_i$', fontsize=16)
plt.ylabel('pdf', fontsize=16)
plt.legend(loc='upper left', fontsize=16)
plt.show()
fig.savefig("U_gtex_cnc.pdf")

In [ ]:
A = odf['A'].values
A_c = odf[odf.index.isin(pc.index)]['A'].values
A_nc = odf[~odf.index.isin(pc.index)]['A'].values
fig=plt.figure()
plt.plot(np.arange(0,len(A)), 1./np.arange(0,len(A)), ls='--', c='g', label='$r^-1$')
#plt.plot(np.arange(0,len(A)), np.sort(A)[::-1]/np.sum(A))
#plt.plot(np.arange(0,len(A_c)), np.sort(A_c)[::-1]/np.sum(A), label='coding genes')
plt.plot(np.arange(0,len(A_nc)), np.sort(A_nc)[::-1]/np.sum(A), label='non coding genes')
plt.xscale('log')
plt.yscale('log')
plt.ylabel('$f_i$', fontsize=18)
plt.xlabel("Rank_i", fontsize=18)
plt.legend(fontsize=18)
fig.savefig("globalZipf_nc.pdf")